# Model Eğitimi

In [2]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

Daha önceden işlemeye hazır hale getirip sakladığımız dosyaları çekiyoruz.

In [3]:
def load_data(path):
    with open( path, 'rb') as handle:
        dataset = pickle.load(handle)
    return dataset

In [4]:
with open('vectors/news_tr_word.pkl', 'rb') as handle:
    word_tfidf = pickle.load(handle)

with open('vectors/news_tr_char.pkl', 'rb') as handle:
    char_tfidf = pickle.load(handle)

In [5]:
df_train = load_data(r"data/news_tr_train_cleaned.pkl")
df_test = load_data(r"data/news_tr_test_cleaned.pkl")

Veri setlerindeki metinleri, kelime ve karakter düzeyinde TF-IDF özelliklerine dönüştürdük ve daha sonra bu özellikleri birleştirdik. Sonuç olarak, her belge için birleşik bir TF-IDF özellik vektörü oluşturduk. Bu vektörleride daha sonraki adımda modelimize girdi olarak vereceğiz.

In [6]:
train_tf_word_transformed = word_tfidf.transform(df_train['clean_content'])
train_tf_char_transformed = char_tfidf.transform(df_train['clean_content'])

test_tf_word_transformed = word_tfidf.transform(df_test['clean_content'])
test_tf_char_transformed = char_tfidf.transform(df_test['clean_content'])

tf_train_features = hstack([train_tf_word_transformed, train_tf_char_transformed])
tf_train_features.shape

(21888, 4000)

Kelime ve karakter düzeyinde TF-IDF özelliklerine dönüştürülen temizlenmiş metinlerden oluşan verileri kullanarak linear çekirdek fonksiyonuna sahip bir SVM modelimizi eğitiyoruz ve bu modele TF-IDF ile dönüştürülmüş metin verileri ve karşılık gelen kategorileri kullanarak metin sınıflandırması için gerekli olan kalıpları ve ilişkileri öğretiyoruz.

In [ ]:
model = svm.SVC(kernel='linear', gamma='auto')
model.fit(tf_train_features, df_train['category'])

Eğitilmiş modelimizi daha sonra girdiler ile kullanmak üzere kaydediyoruz.

In [ ]:
filename ='models/news_tr_classifier_svm_model.sav'
pickle.dump(model, open(filename, 'wb'))

# Model Performansı Değerlendirme

Burada gerçek test verilerimiz ile modelimizin test verilerindeki performansını değerlendirir.

In [ ]:
tf_test_featured = hstack([test_tf_word_transformed, test_tf_char_transformed])

model.score(tf_test_featured, df_test['category'])

Bu kod parçası, test veri setindeki her bir haber metni için:

Kelime ve karakter düzeyinde TF-IDF özelliklerine dönüştürür.
Bu özellikleri birleştirir.
Eğitilmiş model kullanarak haberin kategorisini tahmin eder.
Tahmin edilen kategoriyi ve gerçek kategoriyi ilgili listelere ekler.
Sonuç olarak, y_pred listesi modelin tahminlerini, y_true listesi ise gerçek kategorileri içerir. Bu iki liste daha sonra modelin performansını değerlendirmek için kullanılabilir, örneğin doğruluk, hassasiyet, duyarlılık, F1 skoru gibi metrikler hesaplanabilir.

In [ ]:
y_pred = []
y_true = []

for i, news in enumerate(tqdm(df_test['clean_content'])):
    word_transformed_news = word_tfidf.transform([news])
    char_transformed_news = char_tfidf.transform([news])
    news_featured = hstack([word_transformed_news, char_transformed_news])
    result = model.predict(news_featured)[0]
    y_pred.append(result)
    y_true.append(df_test['category'].iloc[i])

In [ ]:
labels = ["Kültür-Sanat","Ekonomi","Siyaset","Eğitim","Dünya","Spor","Teknoloji","Magazin","Sağlık","Gündem"]

print(classification_report(y_true, y_pred, target_names=labels))

In [ ]:
conf_mat = confusion_matrix(y_true, y_pred)
fig, ax = plt.subplots(figsize=(15,7))
sns.heatmap(conf_mat, annot=True, fmt='d',
            xticklabels=labels, yticklabels=labels)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()